In [14]:
"""本节知识点：
type
元类
ORM
"""

In [ ]:
class Hello (object):
    def hello(self,name = 'world'):
        print('Hello,%s.' % name)

In [1]:
# 下面这句话的含义是：从hello.py 这个模块中导入 Hello 这个类
from hello import Hello #报错为： ModuleNotFoundError: No module named 'hello'
h = Hello()
h.hello()

ModuleNotFoundError: No module named 'hello'

In [6]:
"""
01.可以使用type()函数动态创建一个类
02.metaclass就是元类 => 当我们定义了类以后，就可以根据这个类创建出实例【先定义类，后创建实例】
"""

SyntaxError: invalid syntax (<ipython-input-6-deb7579f4e14>, line 1)

In [8]:
# metaclass是类的模板，所以必须从 type 类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

#注意()中有两个参数，分别是list 和 metaclass 
class MyList(list,metaclass = ListMetaclass):
    pass

L = MyList()
L.add(1)
L

# metaclass 应用最典型的场景是ORM。
# ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表

[1]

In [13]:
class Field(object):
    def __init__(self,name,column_type):
        self.name = name
        self.column_type = column_type
        
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__,self.name)
    
class StringField(Field):

    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

        
        
class IntegerField(Field):

    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')
        
        
        
class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)

    
    
    
class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    # 定义各种操作数据库的方法
    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

    
"""
01.在当前类User的定义中查找metaclass, => 这里的查找操作是固定的吗？【每次都会执行？】
02.如果上一步没有完成，则在父类中继续查找
"""
class User(Model):
    # 定义类的属性到列的映射：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')    
    
    
# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库：
u.save()



Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']
